<a href="https://colab.research.google.com/github/falloatti/HenryProyectoUno/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Modelo de recomendacion**

In [ ]:
#habilito lectura en Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

pip install sidetable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#importacion de librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from pandas_profiling import ProfileReport
import sidetable
from datetime import datetime, timedelta
from sklearn import metrics
import math
from sklearn.metrics import classification_report
import statsmodels.api as sm
from sklearn import linear_model

In [ ]:
#Importo base de datos de plataformas

dfn = pd.read_csv(r'/content/drive/MyDrive/MLOpsReviews/netflix_titles.csv')
dfh = pd.read_csv(r'/content/drive/MyDrive/MLOpsReviews/hulu_titles.csv')
dfa = pd.read_csv(r'/content/drive/MyDrive/MLOpsReviews/amazon_prime_titles.csv')
dfd = pd.read_csv(r'/content/drive/MyDrive/MLOpsReviews/disney_plus_titles.csv')



Inspeccion de formato y contenido de los archivos

In [ ]:
dfn.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
dfh.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...
3,s4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...
4,s5,Movie,Settlers,NaN,NaN,NaN,"October 22, 2021",2021,NaN,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...


In [ ]:
dfa.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [ ]:
dfd.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


In [ ]:
print('Columnas de Netflix:', list(dfn.columns))
print('Columnas de Hulu:', list(dfh.columns))
print('Columnas de Amazon:', list(dfa.columns))
print('Columnas de Disney:', list(dfd.columns))

Columnas de Netflix: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']
Columnas de Hulu: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']
Columnas de Amazon: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']
Columnas de Disney: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']


In [ ]:
# verificar si los datasets tienen las mismas columnas
if set(dfn.columns) == set(dfh.columns) == set(dfa.columns) == set(dfd.columns):
    print('Los 4 datasets tienen las mismas columnas')
else:
    print('Los 4 datasets no tienen las mismas columnas')

Los 4 datasets tienen las mismas columnas


In [ ]:
print('Tamaño de Netflix:', dfn.shape)
print('Tamaño de Hulu:', dfh.shape)
print('Tamaño de Amazon:', dfa.shape)
print('Tamaño de Disney:', dfd.shape)

Tamaño de Netflix: (8807, 12)
Tamaño de Hulu: (3073, 12)
Tamaño de Amazon: (9668, 12)
Tamaño de Disney: (1450, 12)


In [ ]:
#antes de concatenar los archivos voy a agregarles una columna con el nombre de la plaforma por si mas adelante es necesario diferenciarlo, sino lo elimino luego
dfn = dfn.assign(plataforma=pd.Series(['n'] * len(dfn)))
dfh = dfh.assign(plataforma=pd.Series(['h'] * len(dfh)))
dfa = dfa.assign(plataforma=pd.Series(['a'] * len(dfa)))
dfd = dfd.assign(plataforma=pd.Series(['d'] * len(dfd)))

In [ ]:
print('Columnas de Netflix:', list(dfn.columns))
print('Columnas de Hulu:', list(dfh.columns))
print('Columnas de Amazon:', list(dfa.columns))
print('Columnas de Disney:', list(dfd.columns))

Columnas de Netflix: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description', 'plataforma']
Columnas de Hulu: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description', 'plataforma']
Columnas de Amazon: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description', 'plataforma']
Columnas de Disney: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description', 'plataforma']


In [ ]:
dfh.rating.unique() #en esta columna del dataset de hulu vienen valores de 'duration' en rating

array(['TV-MA', nan, 'PG-13', 'R', 'TV-14', 'PG', 'TV-PG', 'NOT RATED',
       'G', 'TV-G', '2 Seasons', 'TV-Y', '93 min', '4 Seasons', 'TV-Y7',
       '136 min', '91 min', '85 min', '98 min', '89 min', '94 min',
       '86 min', '3 Seasons', '121 min', '88 min', '101 min', '1 Season',
       '83 min', '100 min', '95 min', '92 min', '96 min', '109 min',
       '99 min', '75 min', '87 min', '67 min', '104 min', '107 min',
       '84 min', '103 min', '105 min', '119 min', '114 min', '82 min',
       '90 min', '130 min', '110 min', '80 min', '6 Seasons', '97 min',
       '111 min', '81 min', '49 min', '45 min', '41 min', '73 min',
       '40 min', '36 min', '39 min', '34 min', '47 min', '65 min',
       '37 min', '78 min', '102 min', '129 min', '115 min', '112 min',
       'NR', '61 min', '106 min', '76 min', '77 min', '79 min', '157 min',
       '28 min', '64 min', '7 min', '5 min', '6 min', '127 min',
       '142 min', '108 min', '57 min', '118 min', '116 min', '12 Seasons',
       '71 

In [ ]:
dfh['columna_nueva'] = dfh['rating'][pd.notnull(dfh['rating']) & dfh['rating'].str[0].str.isdigit()]


In [ ]:
dfh.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,columna_nueva
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...,h,NaN
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r...",h,NaN
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...,h,NaN
3,s4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...,h,NaN
4,s5,Movie,Settlers,NaN,NaN,NaN,"October 22, 2021",2021,NaN,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...,h,NaN


In [ ]:
dfh.columna_nueva.unique() #en la nueva columna estan solo los que me interesan

array([nan, '2 Seasons', '93 min', '4 Seasons', '136 min', '91 min',
       '85 min', '98 min', '89 min', '94 min', '86 min', '3 Seasons',
       '121 min', '88 min', '101 min', '1 Season', '83 min', '100 min',
       '95 min', '92 min', '96 min', '109 min', '99 min', '75 min',
       '87 min', '67 min', '104 min', '107 min', '84 min', '103 min',
       '105 min', '119 min', '114 min', '82 min', '90 min', '130 min',
       '110 min', '80 min', '6 Seasons', '97 min', '111 min', '81 min',
       '49 min', '45 min', '41 min', '73 min', '40 min', '36 min',
       '39 min', '34 min', '47 min', '65 min', '37 min', '78 min',
       '102 min', '129 min', '115 min', '112 min', '61 min', '106 min',
       '76 min', '77 min', '79 min', '157 min', '28 min', '64 min',
       '7 min', '5 min', '6 min', '127 min', '142 min', '108 min',
       '57 min', '118 min', '116 min', '12 Seasons', '71 min'],
      dtype=object)

In [ ]:
dfh['rating'] = dfh['rating'].where(pd.isna(dfh['rating']) | ~dfh['rating'].astype(str).str[0].str.isdigit(), np.nan)


In [ ]:
dfh.rating.unique() #en 'rating' estan solo los que me interesan

array(['TV-MA', nan, 'PG-13', 'R', 'TV-14', 'PG', 'TV-PG', 'NOT RATED',
       'G', 'TV-G', 'TV-Y', 'TV-Y7', 'NR'], dtype=object)

In [ ]:
# ahora necesito llevar los valores no NaN de la columna nueva a 'duration'
dfh['duration'] = dfh['columna_nueva'].fillna(dfh['duration'])


In [ ]:
dfh = dfh.drop('columna_nueva', axis=1) #elimino la columna provisoria creada


chequeo los demas dataset si tienen el mismo problema

In [ ]:
dfa.rating.unique() #ok

array([nan, '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G',
       'PG', 'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE'], dtype=object)

In [ ]:
dfd.rating.unique() #ok

array(['TV-G', 'PG', 'TV-PG', nan, 'PG-13', 'TV-14', 'G', 'TV-Y7', 'TV-Y',
       'TV-Y7-FV'], dtype=object)

In [ ]:
dfn.rating.unique() #tambien hay errores

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', nan,
       'TV-Y7-FV', 'UR'], dtype=object)

In [ ]:
#implemento misma solucion para plataforma netflix
dfn['columna_nueva'] = dfn['rating'][pd.notnull(dfn['rating']) & dfn['rating'].str[0].str.isdigit()]


In [ ]:
dfn.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,columna_nueva
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",n,NaN
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",n,NaN
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,n,NaN
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",n,NaN
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,n,NaN


In [ ]:
dfn.columna_nueva.unique() #en la nueva columna estan solo los que me interesan

array([nan, '74 min', '84 min', '66 min'], dtype=object)

In [ ]:
dfn['rating'] = dfn['rating'].where(pd.isna(dfn['rating']) | ~dfn['rating'].astype(str).str[0].str.isdigit(), np.nan)


In [ ]:
dfn.rating.unique() #en 'rating' estan solo los que me interesan

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', nan, 'NR', 'TV-Y7-FV', 'UR'], dtype=object)

In [ ]:
# ahora necesito llevar los valores no NaN de la columna nueva a 'duration'
dfn['duration'] = dfn['columna_nueva'].fillna(dfn['duration'])

In [ ]:
dfn = dfn.drop('columna_nueva', axis=1) #elimino la columna provisoria creada


In [ ]:
# verificar si los datasets tienen las mismas columnas
if set(dfn.columns) == set(dfh.columns) == set(dfa.columns) == set(dfd.columns):
    print('Los 4 datasets tienen las mismas columnas')
else:
    print('Los 4 datasets no tienen las mismas columnas')

Los 4 datasets tienen las mismas columnas


In [ ]:
# concateno
dfplataformas = pd.concat([dfn, dfh, dfa, dfd], ignore_index=True)


In [ ]:
# inspecciono tamano
dfplataformas.shape

(22998, 13)

In [ ]:
#importo archivos de Rating
df1 = pd.read_csv(r'/content/drive/MyDrive/MLOpsReviews/ratings/1.csv')
df2 = pd.read_csv(r'/content/drive/MyDrive/MLOpsReviews/ratings/2.csv')
df3 = pd.read_csv(r'/content/drive/MyDrive/MLOpsReviews/ratings/3.csv')
df4 = pd.read_csv(r'/content/drive/MyDrive/MLOpsReviews/ratings/4.csv')
df5 = pd.read_csv(r'/content/drive/MyDrive/MLOpsReviews/ratings/5.csv')
df6 = pd.read_csv(r'/content/drive/MyDrive/MLOpsReviews/ratings/6.csv')
df7 = pd.read_csv(r'/content/drive/MyDrive/MLOpsReviews/ratings/7.csv')
df8 = pd.read_csv(r'/content/drive/MyDrive/MLOpsReviews/ratings/8.csv')


In [ ]:
# inspecciono columnas
print('Columnas de df1:', list(df1.columns))
print('Columnas de df2:', list(df2.columns))
print('Columnas de df3:', list(df3.columns))
print('Columnas de df4:', list(df4.columns))
print('Columnas de df5:', list(df5.columns))
print('Columnas de df6:', list(df6.columns))
print('Columnas de df7:', list(df7.columns))
print('Columnas de df8:', list(df8.columns))


Columnas de df1: ['userId', 'rating', 'timestamp', 'movieId']
Columnas de df2: ['userId', 'rating', 'timestamp', 'movieId']
Columnas de df3: ['userId', 'rating', 'timestamp', 'movieId']
Columnas de df4: ['userId', 'rating', 'timestamp', 'movieId']
Columnas de df5: ['userId', 'rating', 'timestamp', 'movieId']
Columnas de df6: ['userId', 'rating', 'timestamp', 'movieId']
Columnas de df7: ['userId', 'rating', 'timestamp', 'movieId']
Columnas de df8: ['userId', 'rating', 'timestamp', 'movieId']


In [ ]:
# verificar si los datasets tienen las mismas columnas
if set(df1.columns) == set(df2.columns) == set(df3.columns) == set(df4.columns)== set(df5.columns)== set(df6.columns)== set(df7.columns) == set(df8.columns):
    print('Los 8 datasets tienen las mismas columnas')
else:
    print('Los 8 datasets no tienen las mismas columnas')

Los 8 datasets tienen las mismas columnas


In [ ]:
# inspecciono tamano
print('Tamaño de df1:', df1.shape)
print('Tamaño de df2:', df2.shape)
print('Tamaño de df3:', df3.shape)
print('Tamaño de df4:', df4.shape)
print('Tamaño de df5:', df5.shape)
print('Tamaño de df6:', df6.shape)
print('Tamaño de df7:', df7.shape)
print('Tamaño de df8:', df8.shape)



Tamaño de df1: (1500000, 4)
Tamaño de df2: (1500000, 4)
Tamaño de df3: (1500000, 4)
Tamaño de df4: (1500000, 4)
Tamaño de df5: (1500000, 4)
Tamaño de df6: (1500000, 4)
Tamaño de df7: (524289, 4)
Tamaño de df8: (1500000, 4)


In [ ]:
# concateno
dfrating = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], ignore_index=True)


In [ ]:
# inspecciono tamano 
dfrating.shape

(11024289, 4)

**Transformaciones**

Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)


In [ ]:
dfplataformas['id'] = dfplataformas['plataforma'] + dfplataformas['show_id']

In [ ]:
dfplataformas.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,id
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",n,ns1
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",n,ns2
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,n,ns3
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",n,ns4
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,n,ns5


Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”




In [ ]:
# inspecciono valores
print(dfplataformas['rating'].value_counts(dropna=False))


TV-MA        3675
TV-14        3138
R            2154
13+          2117
TV-PG        1654
16+          1547
ALL          1268
18+          1243
NaN          1159
PG-13        1112
PG            881
TV-G          767
TV-Y7         550
TV-Y          462
G             405
7+            385
NR            304
TV-NR         105
UNRATED        33
TV-Y7-FV       19
NC-17           6
AGES_18_        3
UR              3
NOT_RATE        3
AGES_16_        2
16              1
NOT RATED       1
ALL_AGES        1
Name: rating, dtype: int64


In [ ]:
print(dfplataformas['rating'].isnull().sum())

1159


In [ ]:
print(dfplataformas['rating'].value_counts()['G'])


405


In [ ]:
dfplataformas['rating'] = dfplataformas['rating'].fillna('G')

In [ ]:
print(dfplataformas['rating'].value_counts(dropna=False))


TV-MA        3675
TV-14        3138
R            2154
13+          2117
TV-PG        1654
G            1564
16+          1547
ALL          1268
18+          1243
PG-13        1112
PG            881
TV-G          767
TV-Y7         550
TV-Y          462
7+            385
NR            304
TV-NR         105
UNRATED        33
TV-Y7-FV       19
NC-17           6
AGES_18_        3
UR              3
NOT_RATE        3
AGES_16_        2
16              1
NOT RATED       1
ALL_AGES        1
Name: rating, dtype: int64


In [ ]:
print(dfplataformas['rating'].value_counts()['G'])


1564


De haber fechas, deberán tener el formato AAAA-mm-dd

In [ ]:
print(dfplataformas['release_year'].value_counts(dropna=False))


2019    2470
2020    2406
2021    2385
2018    2105
2017    1892
        ... 
1922       2
1926       2
1924       1
1927       1
1928       1
Name: release_year, Length: 101, dtype: int64


In [ ]:
dfplataformas['date_added'] = pd.to_datetime(dfplataformas['date_added'])

In [ ]:
dfplataformas.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,id
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",n,ns1
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",n,ns2
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,n,ns3
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",n,ns4
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,n,ns5


Los campos de texto deberán estar en minúsculas, sin excepciones



In [ ]:
dfplataformas = dfplataformas.applymap(lambda s: s.lower() if type(s) == str else s)


In [ ]:
dfplataformas.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,id
0,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",n,ns1
1,s2,tv show,blood & water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,2021-09-24,2021,tv-ma,2 seasons,"international tv shows, tv dramas, tv mysteries","after crossing paths at a party, a cape town t...",n,ns2
2,s3,tv show,ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",NaN,2021-09-24,2021,tv-ma,1 season,"crime tv shows, international tv shows, tv act...",to protect his family from a powerful drug lor...,n,ns3
3,s4,tv show,jailbirds new orleans,NaN,NaN,NaN,2021-09-24,2021,tv-ma,1 season,"docuseries, reality tv","feuds, flirtations and toilet talk go down amo...",n,ns4
4,s5,tv show,kota factory,NaN,"mayur more, jitendra kumar, ranjan raj, alam k...",india,2021-09-24,2021,tv-ma,2 seasons,"international tv shows, romantic tv shows, tv ...",in a city of coaching centers known to train i...,n,ns5


El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)



In [ ]:
dfplataformas[['duration_int', 'duration_type']] = dfplataformas['duration'].str.split(' ', 1, expand=True)


In [ ]:
dfplataformas.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,id,duration_int,duration_type
0,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",n,ns1,90,min


In [ ]:
#chequeo columnas y tipos de datos
print(dfplataformas.dtypes)

show_id                  object
type                     object
title                    object
director                 object
cast                     object
country                  object
date_added       datetime64[ns]
release_year              int64
rating                   object
duration                 object
listed_in                object
description              object
plataforma               object
id                       object
duration_int             object
duration_type            object
dtype: object


In [ ]:
dfplataformas['duration_int'].unique

<bound method Series.unique of 0         90
1          2
2          1
3          1
4          2
        ... 
22993    108
22994    106
22995    107
22996    112
22997      2
Name: duration_int, Length: 22998, dtype: object>

In [ ]:
# como no me deja pasar la columna 'duration_int' como tipo entero porque hay nulos, reemplazo esos valores nulos por 0, despues vemos el tratamiento.
dfplataformas['duration_int'] = dfplataformas['duration_int'].fillna(0)


In [ ]:
dfplataformas.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,id,duration_int,duration_type
0,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",n,ns1,90,min
1,s2,tv show,blood & water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,2021-09-24,2021,tv-ma,2 seasons,"international tv shows, tv dramas, tv mysteries","after crossing paths at a party, a cape town t...",n,ns2,2,seasons
2,s3,tv show,ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",NaN,2021-09-24,2021,tv-ma,1 season,"crime tv shows, international tv shows, tv act...",to protect his family from a powerful drug lor...,n,ns3,1,season
3,s4,tv show,jailbirds new orleans,NaN,NaN,NaN,2021-09-24,2021,tv-ma,1 season,"docuseries, reality tv","feuds, flirtations and toilet talk go down amo...",n,ns4,1,season
4,s5,tv show,kota factory,NaN,"mayur more, jitendra kumar, ranjan raj, alam k...",india,2021-09-24,2021,tv-ma,2 seasons,"international tv shows, romantic tv shows, tv ...",in a city of coaching centers known to train i...,n,ns5,2,seasons


In [ ]:
dfplataformas.duration_int.unique()

array(['90', '2', '1', '91', '125', '9', '104', '127', '4', '67', '94',
       '5', '161', '61', '166', '147', '103', '97', '106', '111', '3',
       '110', '105', '96', '124', '116', '98', '23', '115', '122', '99',
       '88', '100', '6', '102', '93', '95', '85', '83', '113', '13',
       '182', '48', '145', '87', '92', '80', '117', '128', '119', '143',
       '114', '118', '108', '63', '121', '142', '154', '120', '82', '109',
       '101', '86', '229', '76', '89', '156', '112', '107', '129', '135',
       '136', '165', '150', '133', '70', '84', '140', '78', '7', '64',
       '59', '139', '69', '148', '189', '141', '130', '138', '81', '132',
       '10', '123', '65', '68', '66', '62', '74', '131', '39', '46', '38',
       '8', '17', '126', '155', '159', '137', '12', '273', '36', '34',
       '77', '60', '49', '58', '72', '204', '212', '25', '73', '29', '47',
       '32', '35', '71', '149', '33', '15', '54', '224', '162', '37',
       '75', '79', '55', '158', '164', '173', '181', '185

In [ ]:
dfplataformas['duration_int'] = dfplataformas['duration_int'].astype(int)


In [ ]:
#dfplataformas.to_csv('dfplataformas.csv')

In [ ]:
dfrating.head()

,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663
4,1,5.0,1425941556,as9500


In [ ]:
dfrating.userId.unique()

array([     1,      2,      3, ..., 124378, 124379, 124380])

In [ ]:
for columna in dfplataformas.columns:
    lista_valores_unicos = dfplataformas[columna].unique().tolist()
    print("Valores únicos de la columna", columna, ":", lista_valores_unicos)

Valores únicos de la columna show_id : ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27', 's28', 's29', 's30', 's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's39', 's40', 's41', 's42', 's43', 's44', 's45', 's46', 's47', 's48', 's49', 's50', 's51', 's52', 's53', 's54', 's55', 's56', 's57', 's58', 's59', 's60', 's61', 's62', 's63', 's64', 's65', 's66', 's67', 's68', 's69', 's70', 's71', 's72', 's73', 's74', 's75', 's76', 's77', 's78', 's79', 's80', 's81', 's82', 's83', 's84', 's85', 's86', 's87', 's88', 's89', 's90', 's91', 's92', 's93', 's94', 's95', 's96', 's97', 's98', 's99', 's100', 's101', 's102', 's103', 's104', 's105', 's106', 's107', 's108', 's109', 's110', 's111', 's112', 's113', 's114', 's115', 's116', 's117', 's118', 's119', 's120', 's121', 's122', 's123', 's124', 's125', 's126', 's127', 's128', 's129', 's130', 's131', 's132', 's133', 's13

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
dfrating.columns

Index(['userId', 'rating', 'timestamp', 'movieId'], dtype='object')

In [ ]:
dfrating = dfrating.rename(columns={'rating': 'score'})


In [ ]:
import re

In [ ]:
# contar la cantidad de valores por grupo de las 2 primeras letras del código para asegurar que no haya otras siglas iniciales de plataforma
dfrating['grupo2'] = dfrating['movieId'].apply(lambda x: re.findall('^[a-zA-Z]{2}', x)[0])
conteo2 = dfrating['grupo2'].value_counts()

# mostrar el resultado
print(conteo2)

as    4631850
ns    4224861
hs    1472788
ds     694790
Name: grupo2, dtype: int64


In [ ]:
dfrating = dfrating.rename(columns={'movieId': 'id'})


In [ ]:
dfrating.head()

,userId,score,timestamp,id,grupo2
0,1,1.0,1425941529,as680,as
1,1,4.5,1425942435,ns2186,ns
2,1,5.0,1425941523,hs2381,hs
3,1,5.0,1425941546,ns3663,ns
4,1,5.0,1425941556,as9500,as


In [ ]:
dfrating = dfrating.drop('grupo2', axis=1)


In [ ]:
dfrating.head(1)

,userId,score,timestamp,id
0,1,1.0,1425941529,as680


In [ ]:
# convertir la columna "timestamp" a tipo datetime y mostrar solo el día
dfrating['timestamp'] = pd.to_datetime(dfrating['timestamp'], unit='s').dt.strftime('%Y-%m-%d')

# volver a convertir la columna "timestamp" a tipo datetime
dfrating['timestamp'] = pd.to_datetime(dfrating['timestamp'])

In [ ]:
dfrating.head()

,userId,score,timestamp,id
0,1,1.0,2015-03-09,as680
1,1,4.5,2015-03-09,ns2186
2,1,5.0,2015-03-09,hs2381
3,1,5.0,2015-03-09,ns3663
4,1,5.0,2015-03-09,as9500


In [ ]:
df_movies= pd.merge(dfrating, dfplataformas, on='id')


In [ ]:
df_movies.shape

(11024289, 19)

In [ ]:
df_movies.head()

,userId,score,timestamp,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,duration_int,duration_type
0,1,1.0,2015-03-09,as680,s680,tv show,the english civil war,NaN,robert whelan,NaN,NaT,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,a,1,season
1,583,4.5,2005-04-08,as680,s680,tv show,the english civil war,NaN,robert whelan,NaN,NaT,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,a,1,season
2,765,5.0,1998-03-12,as680,s680,tv show,the english civil war,NaN,robert whelan,NaN,NaT,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,a,1,season
3,2116,3.0,1996-07-01,as680,s680,tv show,the english civil war,NaN,robert whelan,NaN,NaT,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,a,1,season
4,2143,3.0,2015-06-23,as680,s680,tv show,the english civil war,NaN,robert whelan,NaN,NaT,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,a,1,season


In [ ]:
#chequeo columnas y tipos de datos
print(df_movies.dtypes)

userId                    int64
score                   float64
timestamp        datetime64[ns]
id                       object
show_id                  object
type                     object
title                    object
director                 object
cast                     object
country                  object
date_added       datetime64[ns]
release_year              int64
rating                   object
duration                 object
listed_in                object
description              object
plataforma               object
duration_int              int64
duration_type            object
dtype: object


In [ ]:
dfrating.head(1)

,userId,score,timestamp,id
0,1,1.0,2015-03-09,as680


In [ ]:
df_movies.head(1)

,userId,score,timestamp,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,duration_int,duration_type
0,1,1.0,2015-03-09,as680,s680,tv show,the english civil war,NaN,robert whelan,NaN,NaT,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,a,1,season


In [ ]:
dfplataformas.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,id,duration_int,duration_type
0,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",n,ns1,90,min


In [ ]:
# genero valor promedio de pelicula

In [ ]:
# Agrupar los datos por "id" y calcular el promedio de "score"
promedios = dfrating.groupby("id")["score"].mean()
promedios_redondeados = promedios.round(2)
nuevo_datos = pd.DataFrame({'id': promedios_redondeados.index, 'ScoreMedio': promedios_redondeados.values})


In [ ]:
nuevo_datos.head(1)

,id,ScoreMedio
0,as1,3.47


In [ ]:
promedios.loc['as680']

3.5491266375545854

In [ ]:
dfrating.groupby('id')['score'].transform('mean')

0           3.549127
1           3.556886
2           3.604124
3           3.547085
4           3.532158
              ...   
11024284    3.593275
11024285    3.536561
11024286    3.454183
11024287    3.620278
11024288    3.565476
Name: score, Length: 11024289, dtype: float64

In [ ]:
nuevo_datos.shape

(22998, 2)

In [ ]:
nuevo_datos[nuevo_datos['id']=='as680']

,id,ScoreMedio
6445,as680,3.55


In [ ]:
nuevo_datos.shape

(22998, 2)

In [ ]:
# Agrupar los datos por "id" y calcular el promedio de "score"
promedios = dfrating.groupby("id")["score"].mean()
promedios_redondeados = promedios.round(2)
nuevo_datos = pd.DataFrame({'id': promedios_redondeados.index, 'ScoreMedio': promedios_redondeados.values})



In [ ]:
nuevo_datos.head()

,id,ScoreMedio
0,as1,3.47
1,as10,3.44
2,as100,3.61
3,as1000,3.56
4,as1001,3.59


In [ ]:
nuevo_datos.ScoreMedio.unique()

array([3.47, 3.44, 3.61, 3.56, 3.59, 3.6 , 3.51, 3.48, 3.58, 3.45, 3.5 ,
       3.53, 3.41, 3.62, 3.49, 3.52, 3.57, 3.42, 3.54, 3.63, 3.43, 3.64,
       3.55, 3.46, 3.65, 3.67, 3.68, 3.66, 3.39, 3.69, 3.7 , 3.4 , 3.35,
       3.37, 3.38, 3.71, 3.34, 3.72])

In [ ]:
dfrating.score.unique()

array([1. , 4.5, 5. , 4. , 3.5, 2.5, 0.5, 3. , 2. , 1.5])

In [ ]:
df_plataformas_final= pd.merge(dfplataformas, nuevo_datos, on='id')


In [ ]:
df_plataformas_final.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,id,duration_int,duration_type,ScoreMedio
0,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",n,ns1,90,min,3.61


In [ ]:
df_plataformas_final.duration_type.unique()

array(['min', 'seasons', 'season', nan], dtype=object)

In [ ]:
df_plataformas_final.shape

(22998, 17)

In [ ]:
# Contar los valores duplicados
num_duplicados = df_plataformas_final.duplicated().sum()

# Imprimir el número de duplicados
print("Hay", num_duplicados, "valores duplicados en el conjunto de datos.")

Hay 0 valores duplicados en el conjunto de datos.


In [ ]:
df_plataformas_final.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,id,duration_int,duration_type,ScoreMedio
0,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",n,ns1,90,min,3.61


voy a renombrar las variables para que queden como en las consignas

In [ ]:
df_movies.head(1)

,userId,score,timestamp,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,duration_int,duration_type
0,1,1.0,2015-03-09,as680,s680,tv show,the english civil war,NaN,robert whelan,NaN,NaT,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,a,1,season


In [ ]:
#df_plataformas_final.to_csv('df_plataformas_final.csv')

In [ ]:
pip install pandasql


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Modificacion de nombre de las variables para que queden igual a las consultas en fastapi**

In [ ]:
df_plataformas_final.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,id,duration_int,duration_type,ScoreMedio
0,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",n,ns1,90,min,3.61


In [ ]:
df_plataformas_final = df_plataformas_final.rename(columns={'release_year': 'year', 'plataforma': 'platform','ScoreMedio':'scored'})


In [ ]:
df_plataformas_final.head(1)

,show_id,type,title,director,cast,country,date_added,year,rating,duration,listed_in,description,platform,id,duration_int,duration_type,scored
0,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",n,ns1,90,min,3.61


In [ ]:
df_plataformas_final.duration_type.unique()

array(['min', 'seasons', 'season', nan], dtype=object)

In [ ]:
df_plataformas_final['duration_type'] = df_plataformas_final['duration_type'].replace('seasons', 'season')


In [ ]:
df_plataformas_final.duration_type.unique()

array(['min', 'season', nan], dtype=object)

In [ ]:
df_plataformas_final.listed_in.unique()

array(['documentaries', 'international tv shows, tv dramas, tv mysteries',
       'crime tv shows, international tv shows, tv action & adventure',
       ..., 'animation, family, mystery', 'biographical, comedy, drama',
       'action-adventure, animals & nature, animation'], dtype=object)

In [ ]:
reemplazos = {'n': 'netflix', 'd': 'disney', 'h': 'hulu', 'a': 'amazon'}
df_plataformas_final['platform'] = df_plataformas_final['platform'].replace(reemplazos)

In [ ]:
df_plataformas_final.platform.unique()

array(['netflix', 'hulu', 'amazon', 'disney'], dtype=object)

In [ ]:
df_plataformas_final.head(1)

,show_id,type,title,director,cast,country,date_added,year,rating,duration,listed_in,description,platform,id,duration_int,duration_type,scored
0,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",netflix,ns1,90,min,3.61


In [ ]:
nuevo_datos.to_csv('scored.csv')

In [ ]:
#df_plataformas_final.to_csv('df_plataformas_final.csv')

In [ ]:
import pickle as pkl

In [ ]:
df_movies.shape

(11024289, 19)

In [ ]:
dfrating.head(1)

,userId,score,timestamp,id
0,1,1.0,2015-03-09,as680


In [ ]:
df_plataformas_final.head()

,show_id,type,title,director,cast,country,date_added,year,rating,duration,listed_in,description,platform,id,duration_int,duration_type,scored
0,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",netflix,ns1,90,min,3.61
1,s2,tv show,blood & water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,2021-09-24,2021,tv-ma,2 seasons,"international tv shows, tv dramas, tv mysteries","after crossing paths at a party, a cape town t...",netflix,ns2,2,season,3.55
2,s3,tv show,ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",NaN,2021-09-24,2021,tv-ma,1 season,"crime tv shows, international tv shows, tv act...",to protect his family from a powerful drug lor...,netflix,ns3,1,season,3.60
3,s4,tv show,jailbirds new orleans,NaN,NaN,NaN,2021-09-24,2021,tv-ma,1 season,"docuseries, reality tv","feuds, flirtations and toilet talk go down amo...",netflix,ns4,1,season,3.56
4,s5,tv show,kota factory,NaN,"mayur more, jitendra kumar, ranjan raj, alam k...",india,2021-09-24,2021,tv-ma,2 seasons,"international tv shows, romantic tv shows, tv ...",in a city of coaching centers known to train i...,netflix,ns5,2,season,3.59


In [ ]:
movies = pd.merge(dfrating, df_plataformas_final, on='id')


In [ ]:
movies.head(1)

,userId,score,timestamp,id,show_id,type,title,director,cast,country,date_added,year,rating,duration,listed_in,description,platform,duration_int,duration_type,scored
0,1,1.0,2015-03-09,as680,s680,tv show,the english civil war,NaN,robert whelan,NaN,NaT,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,amazon,1,season,3.55


In [ ]:
movies.shape

(11024289, 20)

In [ ]:
movies = movies.drop('description', axis=1)


In [ ]:
movies = movies.drop('show_id', axis=1)


In [ ]:
movies = movies.drop('duration', axis=1)


In [ ]:
movies.cast.unique()

array(['robert whelan', 'ashley bornancin, carter hastings', nan, ...,
       'tom kane, matt lanter, ashley eckstein, james arnold taylor, dee baker, corey burton',
       'brenda wairimu, nick mutuma, catherine kamau, pascal tokodi, patricia kihoro, pierra makena, bridget shighadi, brian ogola, illya frank, aseem sharma, arthur sanya, justin mirichii, maqbul mohammed',
       'russell curry, hunter deno, kai moya, tessa rao, chance perez, shavaughn ruakere, blair strang'],
      dtype=object)

In [ ]:
movies.to_pickle('movies.pkl')